In [16]:
module lc

import Base: unsafe_convert, read, write, close
using Base.Dates: Period, Millisecond

const liblcm = "$(ENV["HOME"])/.julia/v0.5/PyLCM/deps/usr/lib/liblcm.dylib"

type LCM
    pointer::Ptr{Void}
    filedescriptor::RawFD

    LCM() = begin
        pointer = ccall((:lcm_create, liblcm), Ptr{Void}, (Ptr{UInt8},), "")
        filedescriptor = RawFD(ccall((:lcm_get_fileno, liblcm), Cint, (Ptr{Void},), pointer))
        lc = new(pointer, filedescriptor)
        finalizer(lc, close)
        lc
    end
end
unsafe_convert(::Type{Ptr{Void}}, lc::LCM) = lc.pointer

function close(lcm::LCM)
    if lcm.pointer != C_NULL
        ccall((:lcm_destroy, liblcm), Void, (Ptr{Void},), lcm)
        lcm.pointer = C_NULL
    end
end

function publish(lcm::LCM, channel::String, data::Vector{UInt8})
    ccall((:lcm_publish, liblcm), Cint, (Ptr{Void}, Ptr{UInt8}, Ptr{UInt8}, Cuint), channel, data, length(data))
end

filedescriptor(lcm::LCM) = lcm.filedescriptor

immutable CommsT
    utime::Int64
    format::String
    format_version_major::Int32
    format_version_minor::Int32
    data::Vector{UInt8}
end

function write(io::IO, comms::CommsT)
    const signature = UInt8[211, 104, 224, 63, 51, 197, 104, 190]
    write(io, signature)
    write(io, hton(comms.utime))
    write(io, hton(UInt32(length(comms.format) + 1)))
    write(io, comms.format)
    write(io, UInt8(0))
    write(io, hton(comms.format_version_major))
    write(io, hton(comms.format_version_minor))
    write(io, hton(Int32(length(comms.data))))
    write(io, hton.(comms.data))
end

function read(io::IO, ::Type{CommsT})
    signature = read(io, 8)
    @assert signature == UInt8[211, 104, 224, 63, 51, 197, 104, 190]
    utime = Int64(ntoh(read(io, Int64)))
    formatlen = UInt32(ntoh(read(io, UInt32)))
    format = String(read(io, formatlen - 1))
    read(io, 1) # strip off null
    ver_maj = Int32(ntoh(read(io, Int32)))
    ver_min = Int32(ntoh(read(io, Int32)))
    num_bytes = Int32(ntoh(read(io, Int32)))
    data = String(read(io, UInt8, num_bytes))
    CommsT(utime, format, ver_maj, ver_min, data)
end


immutable RecvBuf
    data::Ptr{UInt8}
    data_size::UInt32
    recv_utime::Int64
    lcm::Ptr{Void}
end

function onresponse(rbuf::RecvBuf, channelbytes::Ptr{UInt8}, data::Ptr{Void})
    channel = unsafe_wrap(String, channelbytes)
    @show channel
    msgdata = unsafe_wrap(Vector{UInt8}, rbuf.data, rbuf.data_size)
    msg = read(IOBuffer(msgdata), CommsT)
    @show msg
    return nothing::Void
end

function subscribe(lcm::LCM, channel::String, handler::Function)
    ccall((:lcm_subscribe, liblcm), Ptr{Void}, 
    (Ptr{Void}, Ptr{UInt8}, Ptr{Void}, Ptr{Void}), 
    lcm,
    channel,
    cfunction(onresponse, Void, (Ref{RecvBuf}, Ptr{UInt8}, Ptr{Void})),
    C_NULL)
end

chandle(lcm::LCM) = ccall((:lcm_handle, liblcm), Cint, (Ptr{Void},), lcm)
    
function handle(lcm::LCM)
    fd = filedescriptor(lcm)
    while true
        event = poll_fd(fd, 10; readable=true)
        if event.readable
            chandle(lcm)
            return true
        end
    end
end

function handle(lcm::LCM, timeout::Period)
    timeout_ms = convert(Int, convert(Millisecond, timeout))
    fd = filedescriptor(lcm)
    event = poll_fd(fd, timeout_ms / 1000; readable=true)
    if event.readable
        chandle(lcm)
        return true
    else
        return false
    end
end 

end

lc

In [17]:
lcm = lc.LCM()

lc.LCM(Ptr{Void} @0x00007fa064666b00,RawFD(68))

In [18]:
lc.subscribe(lcm, "HELLO", lc.onresponse)

Ptr{Void} @0x00007fa0640db880

In [20]:
@async while true
    lc.handle(lcm)
end

Task (runnable) @0x00000001127fdf90

channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


In [21]:
1 + 1

2

channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


In [4]:
lc.handleloop(lcm)

Task (runnable) @0x0000000112329d50

channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


In [5]:
2 + 2

4

channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])
channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


In [108]:
lc.handle(lcm)

channel = "HELLO"
msg = lc.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


0

In [109]:
fd = lc.filedescriptor(lcm)

RawFD(176)

In [111]:
event = poll_fd(fd, 1.0; readable=true)
dump(event)

Base.Filesystem.FDEvent
  readable: Bool true
  writable: Bool false
  timedout: Bool false


In [6]:
typeof(event)

Base.Filesystem.FDEvent

In [40]:
b = IOBuffer()

IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1)

In [41]:
data = "{\"hello\": \"world\"}"
msg = lc.CommsT(123456, "my format", 15, 3, data)

lc.CommsT(123456,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])

In [42]:
write(b, msg)

0x0000000000000012

In [43]:
seekstart(b)
encoded = Int.(readbytes(b))
println(encoded)

[211,104,224,63,51,197,104,190,0,0,0,0,0,1,226,64,0,0,0,10,109,121,32,102,111,114,109,97,116,0,0,0,0,15,0,0,0,3,0,0,0,18,123,34,104,101,108,108,111,34,58,32,34,119,111,114,108,100,34,125]


In [44]:
encoded == expected

true

In [37]:
expected = [211,
 104,
 224,
 63,
 51,
 197,
 104,
 190,
 0,
 0,
 0,
 0,
 0,
 1,
 226,
 64,
 0,
 0,
 0,
 10,
 109,
 121,
 32,
 102,
 111,
 114,
 109,
 97,
 116,
 0,
 0,
 0,
 0,
 15,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 18,
 123,
 34,
 104,
 101,
 108,
 108,
 111,
 34,
 58,
 32,
 34,
 119,
 111,
 114,
 108,
 100,
 34,
 125]


60-element Array{Int64,1}:
 211
 104
 224
  63
  51
 197
 104
 190
   0
   0
   0
   0
   0
   ⋮
 111
  34
  58
  32
  34
 119
 111
 114
 108
 100
  34
 125

In [16]:
hton(Int64(1))

72057594037927936

In [3]:
sig = r"\xd3h\xe0?3\xc5h\xbe"

r"\xd3h\xe0?3\xc5h\xbe"